# MongoDB



In [17]:
import pymongo
import requests
from lxml import etree
import re
import time

In [10]:
client = pymongo.MongoClient("localhost", 27017)
# 连接MangoDB 服务

mydb = client['mydb']
# 创建数据库

test = mydb["test"]
# 创建collection (表)



发现并没有建立 mydb数据库和 test表，这是因为 MongoDB 数据库只有插入数据后才会建立。

In [11]:
test.insert_one({'name':'wei', 'sex': 'male', 'grade': '99'})



## 导出 CSV

```shell
mongoexport -d mydb -c test --type=csv -f name,sex,grade -o test.csv
```

## 爬取豆瓣音乐 TOP250 的数据 存储到 MangoDB 里

In [14]:
client = pymongo.MongoClient("localhost", 27017)
mydb = client['mydb']
music = mydb["doubanMusic"]

headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"
}


In [113]:
def get_url(link):
    res = requests.get(link, headers=headers)
    selector = etree.HTML(res.text)
    links = selector.xpath('//div[@class="pl2"]/a[1]/@href')
    return links


In [119]:
def get_info(url):
    res = requests.get(url, headers=headers)
    selector = etree.HTML(res.text)
    name = selector.xpath('//*[@id="wrapper"]/h1/span/text()')[0]
    author = re.findall('表演者:.*?>(.*?)</a>', res.text, re.S)[0].strip()
    
    styles = re.findall('<span class="pl">流派:</span>&nbsp;(.*?)<br', res.text, re.S)
    if len(styles) == 0:
        style = '未知'
    else:
        style = styles[0].strip()
        
    medium = re.findall('<span class="pl">介质:</span>&nbsp;(.*?)<br', res.text, re.S)[0].strip()
    publishing_time = re.findall('<span class="pl">发行时间:</span>&nbsp;(.*?)<br', res.text, re.S)[0].strip()
    
    publishers = re.findall('<span class="pl">出版者:</span>&nbsp;(.*?)<br', res.text, re.S)
    if len(publishers) == 0:
        publisher = '未知'
    else:
        publisher = publishers[0].strip()
    
    
    rate = float(selector.xpath('//*[@id="interest_sectl"]/div/div[2]/strong/text()')[0])

    info = { 'name': name,
            'author': author,
            'style': style,
            'medium': medium,
            'publishing_time': publishing_time,
            'publisher': publisher,
            'rate': rate,
    }

    return info


In [120]:
for i in range(0,250,25):
    link = 'https://music.douban.com/top250?start={}'.format(str(i))
    print(link)
    for url in get_url(link):
        info = get_info(url)
        print(info)
        music.insert_one(info)
        time.sleep(1)

https://music.douban.com/top250?start=0
{'name': 'We Sing. We Dance. We Steal Things.', 'author': 'Jason Mraz', 'style': '民谣', 'medium': 'Audio CD', 'publishing_time': '2008-05-13', 'publisher': 'Atlantic/WEA', 'rate': 9.1}
{'name': 'Viva La Vida', 'author': 'Coldplay', 'style': '摇滚', 'medium': 'CD', 'publishing_time': '2008-06-17', 'publisher': 'Capitol', 'rate': 8.7}
{'name': '华丽的冒险', 'author': '陈绮贞', 'style': '流行', 'medium': 'CD', 'publishing_time': '2005-09-23', 'publisher': '艾迴唱片', 'rate': 8.9}
{'name': '范特西', 'author': '周杰伦', 'style': '流行', 'medium': 'CD', 'publishing_time': '2001-09-14', 'publisher': 'BMG', 'rate': 9.2}
{'name': '後。青春期的詩', 'author': '五月天', 'style': '摇滚', 'medium': 'CD', 'publishing_time': '2008-10-23', 'publisher': '相信音樂', 'rate': 8.8}
{'name': '是时候', 'author': '孙燕姿', 'style': '流行', 'medium': 'CD', 'publishing_time': '2011-03-08', 'publisher': '美妙音乐', 'rate': 8.6}
{'name': 'Lenka', 'author': 'Lenka', 'style': '流行', 'medium': 'Audio CD', 'publishing_time': '2008-

{'name': '菊次郎の夏', 'author': 'Joe Hisaishi', 'style': '原声', 'medium': 'CD', 'publishing_time': '1999-05-19', 'publisher': 'Universal/Polydor', 'rate': 9.4}
{'name': 'Fearless', 'author': 'Taylor Swift', 'style': '流行', 'medium': 'Audio CD', 'publishing_time': '2008-11-11', 'publisher': 'Big Machine', 'rate': 8.4}
{'name': 'Life In Cartoon Motion', 'author': 'Mika', 'style': '流行', 'medium': 'CD', 'publishing_time': '2007-02-05', 'publisher': 'Universal/Island', 'rate': 8.8}
{'name': 'H³M', 'author': '陈奕迅', 'style': '流行', 'medium': 'CD', 'publishing_time': '2009-03-23', 'publisher': '环球', 'rate': 8.5}
{'name': '神秘嘉宾', 'author': '林宥嘉', 'style': '流行', 'medium': 'CD', 'publishing_time': '2008-06-03', 'publisher': '华研', 'rate': 8.4}
{'name': 'Les Choristes', 'author': 'Bruno Coulais', 'style': '原声', 'medium': 'Audio CD', 'publishing_time': '2004-05-03', 'publisher': 'Nonesuch', 'rate': 9.4}
{'name': '赤子', 'author': '范晓萱&amp;100%', 'style': '流行', 'medium': 'CD', 'publishing_time': '2009-08-10',

{'name': 'Time Flies', 'author': '陈奕迅', 'style': '流行', 'medium': 'CD+DVD', 'publishing_time': '2010-03-12', 'publisher': '新藝寶', 'rate': 8.7}
{'name': '失败者的飞翔', 'author': '陈绮贞', 'style': '流行', 'medium': 'CD', 'publishing_time': '2008-07-13', 'publisher': '亚神音乐', 'rate': 8.6}
{'name': 'My Love', 'author': '田馥甄', 'style': '流行', 'medium': 'CD', 'publishing_time': '2011-09-02', 'publisher': '华研唱片', 'rate': 8.0}
{'name': '亲亲', 'author': '梁静茹', 'style': '流行', 'medium': 'CD', 'publishing_time': '2006-10-06', 'publisher': '滚石', 'rate': 8.1}
{'name': '你王菲所以我王菲', 'author': '王菲', 'style': '流行', 'medium': 'CD', 'publishing_time': '2002', 'publisher': 'EMI百代', 'rate': 9.4}
{'name': 'Stranger Under My Skin', 'author': '陈奕迅', 'style': '流行', 'medium': 'CD', 'publishing_time': '2011-02-22', 'publisher': '新艺宝', 'rate': 8.6}
{'name': 'Come Away with Me', 'author': 'Norah Jones', 'style': '爵士', 'medium': 'CD', 'publishing_time': '2002', 'publisher': 'Blue Note Records', 'rate': 8.8}
{'name': '平凡之路', 'autho

{'name': '我爱南京', 'author': '李志', 'style': '民谣', 'medium': 'CD', 'publishing_time': '2009-10-16', 'publisher': '未知', 'rate': 8.6}
{'name': '消失的光年', 'author': '大乔小乔', 'style': '民谣', 'medium': 'CD', 'publishing_time': '2007-07-12', 'publisher': '九洲音像出版公司', 'rate': 8.2}
{'name': '还有别的办法吗', 'author': '范晓萱', 'style': '流行', 'medium': 'CD', 'publishing_time': '2004', 'publisher': '林暐哲音乐社', 'rate': 8.7}
{'name': '日光倾城', 'author': '卡奇社', 'style': '流行', 'medium': 'CD', 'publishing_time': '2007-04-20', 'publisher': '中国音楽家音像出版社', 'rate': 8.1}
{'name': 'A Little Love', 'author': '冯曦妤', 'style': '流行', 'medium': 'CD', 'publishing_time': '2008-11-20', 'publisher': 'Click Music Ltd.', 'rate': 8.4}
https://music.douban.com/top250?start=175
{'name': 'Yellow', 'author': 'Coldplay', 'style': '摇滚', 'medium': 'Audio CD', 'publishing_time': '2000-07-03', 'publisher': 'EMI Int&#39;l', 'rate': 9.3}
{'name': 'Hopes And Fears', 'author': 'Keane', 'style': '摇滚', 'medium': 'Audio CD', 'publishing_time': '2004', 'pub

{'name': '克卜勒', 'author': '孙燕姿', 'style': '流行', 'medium': 'CD', 'publishing_time': '2014-02-27', 'publisher': '环球唱片', 'rate': 8.4}
{'name': 'E=MC²', 'author': 'Mariah Carey', 'style': '流行', 'medium': 'CD', 'publishing_time': '2008-04-16', 'publisher': 'Island', 'rate': 8.5}
{'name': '9 Crimes', 'author': 'Damien Rice', 'style': '民谣', 'medium': 'Audio CD', 'publishing_time': '2006-11-27', 'publisher': 'Warner Classics UK', 'rate': 9.4}
{'name': '如果有一件事是重要的', 'author': '陈珊妮', 'style': '流行', 'medium': 'CD', 'publishing_time': '2008-11-22', 'publisher': '大樂音樂企業社', 'rate': 8.2}
{'name': '赤裸裸', 'author': '郑钧', 'style': '摇滚', 'medium': 'CD', 'publishing_time': '1994', 'publisher': '红星音乐生产社', 'rate': 8.8}
{'name': '阿岳正传', 'author': '张震岳', 'style': '流行', 'medium': 'CD', 'publishing_time': '2004-06-24', 'publisher': '滚石唱片', 'rate': 9.0}
{'name': '3颗猫饼干', 'author': '朱玫玲', 'style': '未知', 'medium': 'CD', 'publishing_time': '2004/04/26', 'publisher': 'Wind Records Co.Ltd. 风潮有声出版有限公司', 'rate': 9.1}
{